In [1]:
# %matplotlib qt
import pickle
import os
from pylab import *
import math
import folium

In [2]:
def loadFile(name):
    with open(name,"rb") as f:
        return pickle.load(f)

In [107]:
nodexy = loadFile('../0_map_data_processed/node_xy.p')
nodell = loadFile('../0_map_data_processed/node_ll.p')
windows = loadFile('../3_windowing_the_map/map_windows/windows.p')
params = loadFile('../3_windowing_the_map/map_windows/data.p')

In [108]:
x_offset = params['x_offset']
y_offset = params['y_offset']
x_window = params['x_window']
y_window = params['y_window']
R = 6378000
print(params)

{'x_offset': 23986546, 'y_offset': 15527555, 'x_window': 100, 'y_window': 100}


In [109]:
print('Number of windows :',len(windows))

Number of windows : 73692


In [110]:
def findFoot(x,y,x1,y1,x2,y2):
    a = y2-y1; b = x1-x2
    c = y1*(x2-x1) - x1*(y2-y1)
    temp = -(a*x+b*y+c)/(a**2+b**2)
    return temp*a+x,temp*b+y

In [115]:
# lat,lon = map(float,input('').split(','))
# ZOOM = 17
# scale = (2**ZOOM)*(256)
# R = 6378137 #scale = SCALE
# siny = math.sin(lat*(pi/180))
# siny = min(max(siny, -0.9999999999),0.99999999999)
# x = scale*(0.5+(lon/360))
# y = scale*(0.5-(math.log((1+siny)/(1-siny))/(4*pi)))
x,y = map(float,input('').split(','))
x_w = (int(x)-x_offset)//x_window
y_w = (int(y)-y_offset)//y_window
mn = math.inf
print('The window corresponding to given point:',x_w,y_w)

24007830.20356039,15558218.58301493
The window corresponding to given point: 212 306


In [157]:
def findEdgeIndex(x,y,p=False):
    x_w = (int(x)-x_offset)//x_window
    y_w = (int(y)-y_offset)//y_window
    w_codes = []
    for i in range(-2,3):
        for j in range(-2,3):
            code = str(x_w+i)+'_'+str(y_w+j)
            if(code not in windows):
                continue
            w_codes.append(code)
    if(len(w_codes) == 0):
        print('*',end=' ')
        return(-2)
    mn = math.inf
    add_edge = []
    for code in w_codes:
        for edge in windows[code]:
            x1,y1 = nodexy[edge[0]]
            x2,y2 = nodexy[edge[1]]
            fx,fy = findFoot(x,y,x1,y1,x2,y2)
            dst1 = math.inf
            if(fx >= min(x1,x2) and fx <= max(x1,x2)):
                dst1 = (x-fx)**2 + (y-fy)**2
                if(dst1 < mn):
                    mn = dst1; loc = [fx,fy]
                    add_edge = edge
            dst2 = (x-x1)**2 + (y-y1)**2
            if(dst2 < mn):
                mn = dst2; loc = [x1,y1]
                add_edge = edge
            dst3 = (x-x2)**2 + (y-y2)**2
            if(dst3 < mn):
                mn = dst3; loc = [x2,y2]
                add_edge = edge
            if(min(dst1,dst2,dst3) < 400):
                building_density[edge[3]] += 1
    if(p):
        print('Given x,y:',x,y)
        print('Location on edge:',loc)
        print('The corresponding edge:',add_edge)
    if(mn < 150):
        return(add_edge[3])
    else:
        return(-1)

In [113]:
# Loading all the necessary files
G = loadFile('../0_map_data_processed/edges_list.p')
building_details = loadFile('../0_map_data_processed/building_details.p')

In [146]:
%%time
G[findEdgeIndex(24007830.20356039,15558218.58301493,True)]

Given x,y: 24007830.20356039 15558218.58301493
Location on edge: [24007827.264201593, 15558216.701748017]
The corresponding edge: ('3309317506', '3309317502', 39.40460976772992, 234511)
Wall time: 0 ns


('3309317506', '3309317502', 39.40460976772992, 234511)

In [158]:
# %%time
building_density = [0]*(len(G))
cnt = 1; invalid = 0
for x,y,a in building_details:
    if(cnt%1000 == 0):
        print(cnt//1000, end=' ')
    cnt += 1
    res = findEdgeIndex(x,y)
    if(res == -2):
        invalid += 1
print('')
print("Error Buildings:",invalid)
print("Done")

1 2 3 4 5 6 7 8 9 10 11 12 13 14 * * * * * * * * 15 16 17 18 19 20 21 22 23 24 25 26 27 * 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 * * * * 63 64 * * * * 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 * * 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 * * * * * * * 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 

In [164]:
print(max(building_density))

136


In [162]:
edge_no = 1233
test = folium.Map(location=nodell[G[edge_no][0]], zoom_start=19)
folium.Marker(nodell[G[edge_no][0]], popup="here").add_to(test)
folium.Marker(nodell[G[edge_no][1]], popup="here").add_to(test)
print("Buidling Density :", building_density[edge_no])
test

Buidling Density : 4


In [163]:
def store(data, name):
    with open('../0_map_data_processed/'+name+'.p','wb') as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
store(building_density, 'building_density')